In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4

In [2]:
from datasets import load_dataset

dataset = load_dataset("keremberke/painting-style-classification", 'full',download_mode="force_redownload")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4493 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1295 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/629 [00:00<?, ? examples/s]

In [3]:
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

training_set = dataset['train']['image']
training_labels = dataset['train']['labels']
training_directory = dataset['train']['image_file_path']

test_set = dataset['test']['image']
test_labels = dataset['test']['labels']
test_directory = dataset['test']['image_file_path']

Train dataset size: 4493
Test dataset size: 629


In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 4493
    })
    validation: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 1295
    })
    test: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 629
    })
})


In [5]:
print(dataset['test']['image_file_path'])

['/storage/hf-datasets-cache/all/datasets/46955122091010-config-parquet-and-info-keremberke-painting-style-2d758b2f/downloads/extracted/507c7e78288103bcbc27b146a28f73c798082456249797808de7d7ae16cee2f6/Abstract_Expressionism/abidin-dino_unknown-title-8_jpg.rf.3da7a4c49128a46920deb198612f99cd.jpg', '/storage/hf-datasets-cache/all/datasets/46955122091010-config-parquet-and-info-keremberke-painting-style-2d758b2f/downloads/extracted/507c7e78288103bcbc27b146a28f73c798082456249797808de7d7ae16cee2f6/Abstract_Expressionism/basil-beattie_imagine-if-1993_jpg.rf.026404e1835b6db0f4059418b2f1cb79.jpg', '/storage/hf-datasets-cache/all/datasets/46955122091010-config-parquet-and-info-keremberke-painting-style-2d758b2f/downloads/extracted/507c7e78288103bcbc27b146a28f73c798082456249797808de7d7ae16cee2f6/Abstract_Expressionism/betty-parsons_bright-day-1966_jpg.rf.90540e521a12e01d3e5567ec809bd200.jpg', '/storage/hf-datasets-cache/all/datasets/46955122091010-config-parquet-and-info-keremberke-painting-styl

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import tf_keras as keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [7]:
def prepare_dataset(example):
    image = example['image']
    # Ensure the image is in the correct format
    image = tf.ensure_shape(image, (None, None, 3))
    # Resize and normalize
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    return image, example['labels']

def create_tf_dataset(hf_dataset):
    return tf.data.Dataset.from_generator(
        lambda: hf_dataset,
        output_signature={
            'image_file_path': tf.TensorSpec(shape=(), dtype=tf.string),
            'image': tf.TensorSpec(shape=(None, None, 3), dtype=tf.uint8),
            'labels': tf.TensorSpec(shape=(), dtype=tf.int64)
        }
    ).map(prepare_dataset)

In [8]:
labels = ['Realism', 'Art_Nouveau_Modern', 'Analytical_Cubism', 'Cubism', 'Expressionism', 'Action_painting', 'Synthetic_Cubism', 'Symbolism', 'Ukiyo_e', 'Naive_Art_Primitivism', 'Post_Impressionism', 'Impressionism', 'Fauvism', 'Rococo', 'Minimalism', 'Mannerism_Late_Renaissance', 'Color_Field_Painting', 'High_Renaissance', 'Romanticism', 'Pop_Art', 'Contemporary_Realism', 'Baroque', 'New_Realism', 'Pointillism', 'Northern_Renaissance', 'Early_Renaissance', 'Abstract_Expressionism']


In [9]:
train_dataset = create_tf_dataset(dataset['train'])
val_dataset = create_tf_dataset(dataset['validation'])
test_dataset = create_tf_dataset(dataset['test'])


In [10]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Create and compile model
num_classes = len(set(dataset['train']['labels']))

In [11]:
feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(244, 244, 3))

In [12]:
feature_extractor_layer.trainable = False

In [27]:
model = keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(27, activation='softmax')
])

# model = tf.keras.Sequential([
#     tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False),
#     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Dense(num_classes, activation='softmax')
# ])


In [28]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15
)

Epoch 1/15
141/141 [==============================] - 38s 230ms/step - loss: 2.0623 - accuracy: 0.4158 - val_loss: 2.5902 - val_accuracy: 0.2355
Epoch 2/15
141/141 [==============================] - 17s 105ms/step - loss: 1.6162 - accuracy: 0.4945 - val_loss: 2.7432 - val_accuracy: 0.2633
Epoch 3/15
141/141 [==============================] - 18s 114ms/step - loss: 1.4543 - accuracy: 0.5422 - val_loss: 2.3679 - val_accuracy: 0.2873
Epoch 4/15
141/141 [==============================] - 18s 115ms/step - loss: 1.2282 - accuracy: 0.6027 - val_loss: 2.6586 - val_accuracy: 0.2656
Epoch 5/15
141/141 [==============================] - 17s 107ms/step - loss: 1.0801 - accuracy: 0.6419 - val_loss: 2.2971 - val_accuracy: 0.3181
Epoch 6/15
141/141 [==============================] - 17s 108ms/step - loss: 0.9332 - accuracy: 0.6853 - val_loss: 2.1841 - val_accuracy: 0.3614
Epoch 7/15
141/141 [==============================] - 19s 115ms/step - loss: 0.7806 - accuracy: 0.7385 - val_loss: 2.4318 - val_ac

In [ ]:
model.save('saved_classification_model')

In [ ]:
!zip saved_classification_nmodel.zip -r saved_classification_model
